<a href="https://colab.research.google.com/github/DarthJkid/FBREF_Analysis/blob/main/Top_Five_Leagues_Player_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Data Scraping

In [3]:
df = pd.read_html('https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats', attrs={'id': 'stats_standard'}) [0]
df

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                     Rk             Player             Nation   
0                     1         Max Aarons            eng ENG   
1                     2         Max Aarons            eng ENG   
2                     3     Rodrigo Abajas             es ESP   
3                     4     James Abankwah             ie IRL   
4                     5  Keyliane Abdallah             fr FRA   
...                 ...                ...                ...   
2634               2534   Martín Zubimendi             es ESP   
2635               2535   Szymon Żurkowski             pl POL   
2636               2536    Martin Ødegaard             no NOR   
2637               2537        Milan Đurić             ba BIH   
2638               2538        Milan Đurić             ba BIH   

     Unnamed: 3_level_0 Unnamed: 4_level_0  Unnamed: 5_level_0  \
                    Pos              Squad                Comp   
0                    DF        Bournemouth  eng Premier League   
1                    MF           Valencia          es La Liga   
2                    DF           Valencia          es La Liga   
3                 DF,MF            Udinese          it Serie A   
4                    FW          Marseille          fr Ligue 1   
...                 ...                ...                 ...   
2634                 MF      Real Sociedad          es La Liga   
2635                 MF             Empoli          it Serie A   
2636                 MF            Arsenal  eng Premier League   
2637                 FW              Monza          it Serie A   
2638                 FW              Parma          it Serie A   

     Unnamed: 6_level_0 Unnamed: 7_level_0 Playing Time         ...  \
                    Age               Born           MP Starts  ...   
0                25-026               2000            3      1  ...   
1                25-026               2000            1      0  ...   
2                21-263               2003            1      1  ...   
3                21-014               2004            6      0  ...   
4                18-300               2006            1      0  ...   
...                 ...                ...          ...    ...  ...   
2634             25-363               1999           20     19  ...   
2635             27-127               1997            1      0  ...   
2636             26-044               1998           15     14  ...   
2637             34-253               1990           18     13  ...   
2638             34-253               1990            1      1  ...   

     Per 90 Minutes                                                       \
                Ast   G+A  G-PK G+A-PK    xG   xAG xG+xAG  npxG npxG+xAG   
0              0.00  0.00  0.00   0.00  0.00  0.00   0.00  0.00     0.00   
1              0.00  0.00  0.00   0.00  0.00  0.00   0.00  0.00     0.00   
2              0.00  0.00  0.00   0.00  0.10  0.00   0.10  0.10     0.10   
3              0.00  0.00  0.00   0.00  0.06  0.00   0.06  0.06     0.06   
4              0.00  0.00  0.00   0.00  0.00  0.00   0.00  0.00     0.00   
...             ...   ...   ...    ...   ...   ...    ...   ...      ...   
2634           0.05  0.11  0.05   0.11  0.08  0.04   0.12  0.08     0.12   
2635           0.00  0.00  0.00   0.00  0.00  0.00   0.00  0.00     0.00   
2636           0.23  0.30  0.00   0.23  0.22  0.18   0.40  0.16     0.34   
2637           0.07  0.35  0.28   0.35  0.21  0.08   0.28  0.21     0.28   
2638           0.00  0.00  0.00   0.00  0.00  0.00   0.00  0.00     0.00   

     Unnamed: 37_level_0  
                 Matches  
0                Matches  
1                Matches  
2                Matches  
3                Matches  
4                Matches  
...                  ...  
2634             Matches  
2635             Matches  
2636             Matches  
2637             Matches  
2638             Matches  

[2639 rows x 38 columns]

# Data Cleaning and Preprocessing


In [16]:
#df.columns = ['_'.join(str(c) for c in col).strip() for col in df.columns.values]
#df.head()
# Reset column names to a simple range
df.columns = range(df.shape[1])

# Apply the desired naming logic
df.columns = ['_'.join(str(c) for c in col).strip() if isinstance(col, tuple) else str(col).strip() for col in df.columns]
df.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,Playing Time MP,Playing Time Starts,Playing Time Min,Playing Time 90s,Performance Gls,Performance Ast,Performance G+A,Performance G-PK,Performance PK,Performance PKatt,Performance CrdY,Performance CrdR,Expected xG,Expected npxG,Expected xAG,Expected npxG+xAG,Progression PrgC,Progression PrgP,Progression PrgR,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes G+A,Per 90 Minutes G-PK,Per 90 Minutes G+A-PK,Per 90 Minutes xG,Per 90 Minutes xAG,Per 90 Minutes xG+xAG,Per 90 Minutes npxG,Per 90 Minutes npxG+xAG,Matches
0,1,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,25-039,2000,3,1,86,1.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1,8,3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
1,2,Max Aarons,eng ENG,MF,Valencia,es La Liga,25-039,2000,1,0,31,0.3,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,2,3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
2,3,Rodrigo Abajas,es ESP,DF,Valencia,es La Liga,21-276,2003,1,1,65,0.7,0,0,0,0,0,0,1,0,0.1,0.1,0.0,0.1,3,2,3,0.00,0.00,0.00,0.00,0.00,0.10,0.00,0.10,0.10,0.10,Matches
3,4,James Abankwah,ie IRL,"DF,MF",Udinese,it Serie A,21-027,2004,6,0,88,1.0,0,0,0,0,0,0,1,0,0.1,0.1,0.0,0.1,3,4,1,0.00,0.00,0.00,0.00,0.00,0.06,0.00,0.06,0.06,0.06,Matches
4,5,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,18-313,2006,1,0,3,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches


In [32]:
#Finalising Column names
finalised_columns = []
for col in df.columns:
  if 'Playing Time' in col:
    new_col = col.split()[-1]
  elif 'Performance' in col:
    new_col = col.split()[-1]
  elif 'Expected' in col:
    new_col = col.split()[-1]
  elif 'Progression' in col:
    new_col = col.split()[-1]
  elif 'Per 90' in col:
    new_col = col.split()[-1]
  elif 'Pos' in col:
    new_col = 'Position'
  elif 'Nation' in col:
    new_col = 'Nationality'
  elif 'Age' in col:
    new_col = 'Age'
  elif 'Player' in col:
    new_col = 'Name'
  elif 'Squad' in col:
    new_col = 'Team'
  elif 'Comp' in col:
    new_col = 'League'
  elif 'MP' in col:
    new_col = 'Matches Played'
  elif 'Starts' in col:
    new_col = 'Matches Started'
  elif '90s' in col:
    new_col = '90 Minutes Played'
  elif 'Gls' in col:
    new_col = 'Goals'
  elif 'Ast' in col:
    new_col = 'Assists'
  elif 'G+A' in col:
    new_col = 'Goals + Assists'
  elif 'G-PK' in col:
    new_col = 'Goals - Penalty Kick'
  elif 'PK' in col:
    new_col = 'Penalty Kick'
  elif 'PKatt' in col:
    new_col = 'Penalty Kick Attempts'
  elif 'CrdY' in col:
    new_col = 'Yellow Cards'
  elif 'CrdR' in col:
    new_col = 'Red Cards'
  else:
    new_col = col
  finalised_columns.append(new_col)

df.columns = finalised_columns
df.head()

,Rk,Name,Nationality,Position,Team,League,Age,Born,Matches Played,Matches Started,Min,90 Minutes Played,Goals,Assists,Goals + Assists,Goals - Penalty Kick,Penalty Kick,Penalty Kick,Yellow Cards,Red Cards,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,Goals,Assists,Goals + Assists,Goals - Penalty Kick,Goals + Assists,xG,xAG,xG+xAG,npxG,npxG+xAG,Matches
0,1,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,25-026,2000,3,1,86,1.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1,8,3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
1,2,Max Aarons,eng ENG,MF,Valencia,es La Liga,25-026,2000,1,0,31,0.3,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,2,3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
2,3,Rodrigo Abajas,es ESP,DF,Valencia,es La Liga,21-263,2003,1,1,65,0.7,0,0,0,0,0,0,1,0,0.1,0.1,0.0,0.1,3,2,3,0.00,0.00,0.00,0.00,0.00,0.10,0.00,0.10,0.10,0.10,Matches
3,4,James Abankwah,ie IRL,"DF,MF",Udinese,it Serie A,21-014,2004,6,0,88,1.0,0,0,0,0,0,0,1,0,0.1,0.1,0.0,0.1,3,4,1,0.00,0.00,0.00,0.00,0.00,0.06,0.00,0.06,0.06,0.06,Matches
4,5,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,18-300,2006,1,0,3,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
